In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Mar 14 09:51:39 2022

@author: Malhar
"""

# -*- coding: utf-8 -*-

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_validate
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
#import graphviz
#uses the imbalanced learn library to correct imbalanced data

#import the data
df = pd.read_csv("D:\\Malhar\\Anaconda3\\Malhar\\Data\\EUR USD 1H Data Preparation\\EURUSD1H_Cleaned.csv")
#Some Simple metrics on how many True/False values occur for Buy/Sell
#print(df['Buy'].value_counts())
#print(df['Sell'].value_counts())
#print(df.shape)
#Shows Buy Signal 15.72% of the time, Sell Signal 15.54% of the time. We will need to balance the data


#Convert the Buy/Sell columns to numbers, since these are not scaled
df[['Buy','Sell']] = df[['Buy','Sell']] * 1


#There are a lot of columns, so making a list to slice the df in the future is quicker
col_x = ['Volume', 'Month', 'Day', 'Hour', 'Change', 'Range', 'MA5', 'MA7', 'MA9', 'MA12', 'MA14', 'MA21', 'MA24', 'MA50', 'MA100', 'MA120', 'MA200', 'Range5', 'Range7', 'Range9', 'Range12', 'Range14', 'Range21', 'Range24', 'Range50', 'Range100', 'Range120', 'Range200', 'HH2', 'LL2', 'HH3', 'LL3', 'HH4', 'LL4', 'HH5', 'LL5', 'HH6', 'LL6', 'HH7', 'LL7', 'HH8', 'LL8', 'HH9', 'LL9', 'HH10', 'LL10', 'HH11', 'LL11', 'HH12', 'LL12']

    
#x are the independant variables, y is the dependant variable
#b is for a buy outcome, and s for selll outcomes
#although xb and xs are the same, they are done as seperate variables, so they match with the respective y samples with the oversampler

xb_presampled = df[col_x]
xs_presampled = df[col_x]
yb_presampled = df['Buy']
ys_presampled =  df['Sell']


'''
#I realised an error in my Logisitic regression notebook on this dataset. I oversampled the data, then split it.
#You should only oversample the training set, not the test set. I have corrected this this time around
#(I've left it as an error on the old notebook, to see if anyone spots it!)
'''
#Split test and train data for Buy and Sell
xbtrain_presampled, xbtest, ybtrain_presampled, ybtest = train_test_split(
		xb_presampled, yb_presampled, test_size = 0.25, random_state = 0)
xstrain_presampled, xstest, ystrain_presampled, ystest = train_test_split(
		xs_presampled, ys_presampled, test_size = 0.25, random_state = 0)


#Resample to balance, using unbalanced learn, ROS algorithim [faster than SMOTE]
xbtrain, ybtrain = RandomOverSampler().fit_resample(xbtrain_presampled, ybtrain_presampled)
xstrain, ystrain = RandomOverSampler().fit_resample(xstrain_presampled, ystrain_presampled)


#Scale data and make non numeric data numeric
sc_xb = StandardScaler()
xbtrain = sc_xb.fit_transform(xbtrain)
xbtest = sc_xb.transform(xbtest)

sc_xs = StandardScaler()
xstrain = sc_xs.fit_transform(xstrain)
xstest = sc_xs.transform(xstest)
'''
#Train Decision Tree model for Buy and sell
dt_buy = DecisionTreeClassifier(max_depth=70, class_weight={0:1,1:19})
dt_sell = DecisionTreeClassifier(max_depth=70,class_weight={0:1,1:19})
dt_buy.fit(xbtrain,ybtrain)
dt_sell.fit(xstrain,ystrain)

# depth of the decision tree
print('Depth of the Buy Decision Tree :', dt_buy.get_depth())
print('Depth of the Sell Decision Tree :', dt_sell.get_depth())

# Precision Score on training dataset
buy_prediction = dt_buy.predict(xbtrain)
Precision_buy = precision_score(ybtrain,buy_prediction)
print('Precision score of Buy on training dataset : ', Precision_buy)
sell_prediction = dt_sell.predict(xstrain)
Precision_sell = precision_score(ystrain,sell_prediction)
print('Precision score of Sell on training dataset : ', Precision_sell)

# Accuracy Score on training dataset
Accuracy_buy = accuracy_score(ybtrain,buy_prediction)
print('Accuracy score of Buy on test dataset : ', Accuracy_buy)
Accuracy_sell = accuracy_score(ystrain,sell_prediction)
print('Accuracy score of Sell on test dataset : ', Accuracy_sell)

# Precision Score on test dataset
buy_prediction2 = dt_buy.predict(xbtest)
Precision_buy2 = precision_score(ybtest,buy_prediction2)
print('Precision score of Buy on test dataset : ', Precision_buy2)
sell_prediction2 = dt_sell.predict(xstest)
Precision_sell2 = precision_score(ystest,sell_prediction2)
print('Precision score of Sell on test dataset : ', Precision_sell2)

# Accuracy Score on test dataset
Accuracy_buy2 = accuracy_score(ybtest,buy_prediction2)
print('Accuracy score of Buy on test dataset : ', Accuracy_buy2)
Accuracy_sell2 = accuracy_score(ystest,sell_prediction2)
print('Accuracy score of Sell on test dataset : ', Accuracy_sell2)
'''
'''
#we used this code to get optimised values for max_depth via cross validation, using Stratified Kfolds
Dict = {}
for n in range(1,30):
    kfold_scores = cross_validate(DecisionTreeClassifier(max_depth=n),xbtrain,ybtrain, scoring='precision', cv= StratifiedKFold(n_splits=5))
    Dict[n] = kfold_scores['test_score'].mean()
print(Dict)

Dict2 = {}
for n in range(1,30):
    kfold_scores = cross_validate(DecisionTreeClassifier(max_depth=n),xstrain,ystrain, scoring='precision', cv= StratifiedKFold(n_splits=5))
    Dict[n] = kfold_scores['test_score'].mean()
print(Dict2)
'''

#The best max depth for precision is 5 in both cases. A similar code could be used for all parameters, but the precision is quite low, this may not be worthwhile
dt_buy = DecisionTreeClassifier(class_weight={0:1,1:19})
dt_sell = DecisionTreeClassifier(class_weight={0:1,1:19})
dt_buy = dt_buy.fit(xbtrain,ybtrain)
dt_sell = dt_sell.fit(xstrain,ystrain)

buy_prediction2 = dt_buy.predict(xbtest)
Precision_buy2 = precision_score(ybtest,buy_prediction2)
print('Precision score of Buy on test dataset : ', Precision_buy2)
sell_prediction2 = dt_sell.predict(xstest)
Precision_sell2 = precision_score(ystest,sell_prediction2)
print('Precision score of Sell on test dataset : ', Precision_sell2)
#there was an average precision score of just over 0.3 when cross validating the test set. However, in reality..
#the precision scores have come back very low with a max_depth of 5. In reality, the model is still poorly fit

#Lets graph both these results up anyway
plt.figure(figsize=(50,30))
plot_tree(dt_buy, max_depth=5, fontsize = 10)
plot_tree(dt_sell, max_depth=5, fontsize = 10)
